In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import copy
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
scoredict = {}
num = list(range(1,15))
for n in num:
    x = int(pow(2,n))
    scoredict[x] = (n-1)*x

In [ ]:
class Game:
    def __init__(self,dim=4,p=2):
        self.dim = dim
        self.mat = np.zeros((dim,dim),dtype=int)
        self.old_mat = None
        self.game_over = False
        while(p):
            x = int(random.randint(0,dim-1))
            y = int(random.randint(0,dim-1))
            self.mat[x][y] = 2
            p-=1
    def __init__(self,mat):
        self.dim = len(mat)
        self.mat = mat.copy()
        self.old_mat = None
        self.game_over = False
    def fill(self):
        e = list(zip(*np.where(self.mat == 0)))
        if(len(e)<1):
            self.game_over = True
            return -1
        x,y = e[random.randint(0,len(e)-1)]
        if(random.randint(0,1)==1):
            self.mat[x][y] = 2
        else:
            self.mat[x][y] = 4
        return 0
    def horizontalmove(self):
        old_mat = self.mat.copy()
        for irow in range(len(self.mat)):
            row = self.mat[irow]
            i = 0
            pi = -1
            while i < (len(row)):
                if(row[i]!=0):
                    if(pi==-1):
                        pi = i
                    else:
                        if(row[pi]==row[i]):
                            row[pi]*=2
                            row[i] = 0
                            pi = -1
                        else:
                            pi = i
                i+=1
            nr = np.zeros(self.dim,dtype=int)
            ind = 0
            for v in row:
                if(v!=0):
                    nr[ind]=v
                    ind+=1
            self.mat[irow] = nr.copy()
        if(np.sum(abs(self.mat-old_mat))==0):
#             print("invalid move")
            return 0
        else:
            self.old_mat = old_mat
            return 1
    def play(self,move):
        success = 0
        if(move=='a'):
            success=self.horizontalmove()
            if(success):
                self.fill()
        if(move=='d'):
            self.mat = np.flip(self.mat,axis=1)
            success=self.horizontalmove()
            if(success):
                self.old_mat =  np.flip(self.old_mat,axis=1)
                self.fill()
            self.mat = np.flip(self.mat,axis=1)
        if(move=='w'):
            self.mat = self.mat.T
            success=self.horizontalmove()
            if(success):
                self.old_mat = self.old_mat.T
                self.fill()
            self.mat = self.mat.T
        if(move=='s'):
            self.mat = np.flip(self.mat.T,axis=1)
            success=self.horizontalmove()
            if(success):
                self.old_mat = np.flip(self.old_mat,axis=1).T
                self.fill()
            self.mat = np.flip(self.mat,axis=1).T
        if(move=='r'):
            if(self.old_mat is None):
                success=0
#                 print("Invalid move!")
            else:
                success = 1
                self.mat = self.old_mat
                self.old_mat = None
        return (success,self.game_over,self.score())
    def score(self):
        val = 0
        for row in self.mat:
            for c in row:
                if(c!=0):
                    val += scoredict[c]
        return val
    def __str__(self):
        l = ""
        for row in self.mat:
            for val in row:
                l += str(val).ljust(5)
            l+="\n"
        return l

In [ ]:
g = Game()
print(g)
g.play('w')
print(g)

g.play('r')
print(g)

In [ ]:
class Node:
    def __init__(self,game,move=""):
        self.game = copy.deepcopy(game)
        self.move = move
        self.game_over = self.game.game_over
        self.score = self.game.score()
        self.children = None
    def expand(self,height):
        if(height<=0 or self.game_over):
            return
        if(self.children is None):
            self.children = []
            moves = ['a','s','d','f']
            for move in moves:
                newgame = copy.deepcopy(self.game)
                (success, game_over, score) = newgame.play(move)
                if(success):
                    newNode = Node(newgame,move)
                    newNode.expand(height-1)
                    self.children.append(copy.deepcopy(newNode))
        else:
            for child in self.children:
                child.expand(height-1)
    def topscores(self,n):
        if(self.children is None):
            return [self.score]
        else:
            scores = []
            for child in self.children:
                scores += child.topscores(n)
            if(len(scores)>n):
                scores = scores[:n]
            return scores

In [ ]:
class Tree:
    def __init__(self,root,depth=6,ntop=3):
        self.pureRoot = root
        self.root = root
        self.depth = depth
        self.ntop = ntop
    def playgame(self,turns):
        turn_num = 0
        while(turn_num < turns):
            self.root.expand(self.depth)
            # CHECK TOP-N SCORE
            avg = -1
            prefnew = None
            
            for child in self.root.children:
                scores = child.topscores(self.ntop)
                if(len(scores)>0):
                    avgscore = sum(scores)/len(scores)
                    if(avgscore>avg):
                        avg = avgscore
                        prefnew = child
            if(prefnew is None):
                break
            else:
                self.root.children = [prefnew]
                self.root = prefnew
            turn_num += 1
            print("Turn Number: {}, Score: {}".format(turn_num,self.root.score))
        print("Game ended with score = "+str(self.root.score))
    def printgame(self):
        cnode = self.pureRoot
        while(cnode != self.root):
            print(cnode.move,cnode.score)
            print(cnode.game)
            cnode = cnode.children[0]
    def printmoves(self):
        cnode = self.pureRoot
        while(cnode != self.root):
            print(cnode.move)
            cnode = cnode.children[0]

In [ ]:
root = Node(Game(x))

In [ ]:
T = Tree(root)

In [ ]:
T.playgame(100)

In [ ]:
[1616,2068,2172,2012,1532,]